---
## Step 0: Load The Data

In [1]:
import pickle
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split


import glob
filelist = glob.glob('crowdai/*/*.jpg')

X = np.array([np.array((Image.open(fname)).resize((64,64))) for fname in filelist])



y = np.array([int(''.join(fname[10:12])) for fname in filelist])



from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X, y = shuffle(X, y)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state=0)
X_train , X_valid , y_train,  y_valid = train_test_split(X_train, y_train, test_size=0.15,random_state=0)

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [3]:


# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of validation examples
n_validation = len(X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_valid))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 15834
Number of testing examples = 3288
Image data shape = (64, 64, 3)
Number of classes = 38


In [4]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
import numpy as np

X_train=np.average(X_train,axis=3,weights=[0.299,0.587,0.114])
X_valid=np.average(X_valid,axis=3,weights=[0.299,0.587,0.114])
X_test=np.average(X_test,axis=3,weights=[0.299,0.587,0.114])



X_train=(X_train-128)/128
X_valid=(X_valid-128)/128
X_test=(X_test-128)/128
X_train=X_train.reshape(n_train, 64, 64,1)
X_test=X_test.reshape(n_test, 64, 64,1)
X_valid=X_valid.reshape(n_validation, 64, 64,1)

# X_train,X_validation,y_train,y_validation=train_test_split(X_train,y_train,test_size=0.2,random_state=0)


### Model Architecture

In [5]:
### Define your architecture here.
### Feel free to use as many code cells as needed.
import tensorflow as tf
import numpy as np
EPOCHS = 20
BATCH_SIZE = 128

from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    #keep_prob=0.6
    
    
   
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W=tf.Variable(tf.truncated_normal([5,5,1,6],mean=mu,stddev=sigma),name="Pholder3")
    conv1_B=tf.Variable(tf.zeros([6]),name="Pholder4")
    conv1=tf.nn.conv2d(x,conv1_W,[1,1,1,1],'VALID',name='tagget_conv')+conv1_B
    

    conv1=tf.nn.relu(conv1,name="Pholder7")

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1=tf.nn.max_pool(conv1,[1,2,2,1],[1,2,2,1],'VALID',name="Pholder6")

    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W=tf.Variable(tf.truncated_normal([5,5,6,16],mean=mu,stddev=sigma,name="Pholder8"),name="Pholder5")
    conv2_B=tf.Variable(tf.zeros([16]),name="Pholder9")
    conv2=tf.nn.conv2d(conv1,conv2_W,[1,1,1,1],'VALID',name="Pholder10")+conv2_B
    

    conv2=tf.nn.relu(conv2)

    conv2=tf.nn.max_pool(conv2,[1,2,2,1],[1,2,2,1],'VALID')


    fc0=flatten(conv2)
    

    fc1_W=tf.Variable(tf.truncated_normal(shape=(2704,120),mean=mu,stddev=sigma))
    fc1_B=tf.Variable(tf.zeros(120))
    fc1=tf.matmul(fc0,fc1_W)+fc1_B
    

    fc1=tf.nn.relu(fc1)
    fc1=tf.nn.dropout(fc1,keep_prob)


    fc2_W=tf.Variable(tf.truncated_normal(shape=(120,84),mean=mu,stddev=sigma))
    fc2_B=tf.Variable(tf.zeros(84))
    fc2=tf.matmul(fc1,fc2_W)+fc2_B
    

    fc2=tf.nn.relu(fc2)
    fc2=tf.nn.dropout(fc2,keep_prob)

 
    fc3_W=tf.Variable(tf.truncated_normal(shape=(84,38),mean=mu,stddev=sigma))
    fc3_B=tf.Variable(tf.zeros(38))
    logits=tf.matmul(fc2,fc3_W,name="logit")+fc3_B
    print(logits)
    
    return logits

### Train, Validate and Test the Model

In [6]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.
x = tf.placeholder(tf.float32, (None, 64, 64, 1), name="Pholder0")
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 38)
keep_prob=tf.placeholder(tf.float32,name="Pholder11")

In [7]:
rate = 0.00098

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits,name="softmax")
loss_operation = tf.reduce_mean(cross_entropy,name="target_conv")
optimizer = tf.train.AdamOptimizer(learning_rate = rate,name="adam_optimized")
training_operation = optimizer.minimize(loss_operation)

Tensor("add_4:0", shape=(?, 38), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    #loss_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:1})
        #loss=sess.run(loss_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
        #loss_accuracy += (loss * len(batch_x))
    return total_accuracy / num_examples

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            #print(batch_y.shape)
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y,keep_prob:0.6})
        training_accuracy = evaluate(X_train, y_train)    
        validation_accuracy = evaluate(X_valid, y_valid)
        
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")


Training...

EPOCH 1 ...
Train Accuracy = 0.354
Validation Accuracy = 0.332

EPOCH 2 ...
Train Accuracy = 0.455
Validation Accuracy = 0.434

EPOCH 3 ...
Train Accuracy = 0.505
Validation Accuracy = 0.491

EPOCH 4 ...
Train Accuracy = 0.541
Validation Accuracy = 0.522

EPOCH 5 ...
Train Accuracy = 0.565
Validation Accuracy = 0.537

EPOCH 6 ...
Train Accuracy = 0.601
Validation Accuracy = 0.574

EPOCH 7 ...
Train Accuracy = 0.608
Validation Accuracy = 0.569

EPOCH 8 ...
Train Accuracy = 0.643
Validation Accuracy = 0.600

EPOCH 9 ...
Train Accuracy = 0.669
Validation Accuracy = 0.618

EPOCH 10 ...
Train Accuracy = 0.688
Validation Accuracy = 0.626

EPOCH 11 ...
Train Accuracy = 0.698
Validation Accuracy = 0.636

EPOCH 12 ...
Train Accuracy = 0.722
Validation Accuracy = 0.650

EPOCH 13 ...
Train Accuracy = 0.727
Validation Accuracy = 0.648

EPOCH 14 ...
Train Accuracy = 0.748
Validation Accuracy = 0.665

EPOCH 15 ...
Train Accuracy = 0.764
Validation Accuracy = 0.667

EPOCH 16 ...
Train Ac

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

In [ ]:
import tensorflow as tf
meta_path = './lenet.meta' # Your .meta file\n",
output_node_names = ['logit']    # Output nodes\n",

with tf.Session() as sess:

 # Restore the graph\n",
    saver = tf.train.import_meta_graph(meta_path)

   # Load weights\n",
    saver.restore(sess,tf.train.latest_checkpoint('.'))

 # Freeze the graph\n",
    frozen_graph_def = tf.graph_util.convert_variables_to_constants(sess,sess.graph_def,output_node_names)
# Save the frozen graph\n",
    with open('output_graph.pb', 'wb') as f:
        f.write(frozen_graph_def.SerializeToString())

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

test_images=[]
image_labels=[0,1,2,3,4,5,6,7,8,9]
path='./test_data/'
for image in os.listdir(path):
    print(image)
    image_path=cv2.imread(path+image)
    image=cv2.resize(image_path,(64,64))
#     image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    test_images.append(image)
test_image_array=np.array(test_images)


In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
filelist = glob.glob('crowdai/c_36/*.jpg')
# load the image with imread()

for i in filelist:
    imageSource = i
    img = cv2.imread(imageSource)
 
# copy image to display all 4 variations
    horizontal_img = img.copy()
    vertical_img = img.copy()
    both_img = img.copy()
 
# flip img horizontally, vertically,
# and both axes with flip()
    horizontal_img = cv2.flip( img, 0 )
    vertical_img = cv2.flip( img, 1 )
    both_img = cv2.flip( img, -1 )
 
# display the images on screen with imshow()
    path=i.split('.')
    path=''.join(path[:len(path)-1])
    cv2.imwrite( path+"H.jpg", horizontal_img )
  
    cv2.imwrite(path+"V.jpg" , vertical_img )
  
    cv2.imwrite(path+"B.jpg", both_img )
 

In [ ]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
filelist = glob.glob('crowdai/c_09/*.jpg')
# load the image with imread()

for i in filelist:
    img = cv2.imread(i)
    num_rows, num_cols = img.shape[:2]

    rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2), 30, 1)
    img_rotation = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
    path=i.split('.')
    path=''.join(path[:len(path)-1])
    cv2.imwrite(path+"Rotate.jpg", img_rotation)


In [ ]:

test_image_array=np.average(test_image_array,axis=3,weights=[0.299,0.587,0.114])
test_image_array=(test_image_array-128)/128
test_image_array=np.reshape(test_image_array,(10,64,64,1))

In [ ]:
import tensorflow as tf
from tensorflow.contrib.lite.toco.python.toco_wrapper import main
print(tf.__version__)

### Analyze Performance

In [ ]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver_test=tf.train.import_meta_graph('lenet.meta')
    saver_test.restore(sess,tf.train.latest_checkpoint('.'))
    test_accuracy=evaluate(test_image_array,image_labels)
    print("Test Accuracy={:.3f}".format(test_accuracy))

In [ ]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            input_map=None, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

# We use our "load_graph" function
graph = load_graph("./optimized_graph.pb")

# We can verify that we can access the list of operations in the graph
for op in graph.get_operations():
    print(op.name)
    print(op.values())# <--- printing the operations snapshot below
    # prefix/Placeholder/inputs_placeholder
    # ...
    # prefix/Accuracy/predictions

# We access the input and output nodes
x1 = graph.get_tensor_by_name('prefix/Pholder0:0')
y1 = graph.get_tensor_by_name('prefix/logit:0')
keep_probab=graph.get_tensor_by_name('prefix/Pholder11:0')

# We launch a Session
with tf.Session(graph=graph) as sess:

    test_features = test_image_array
        # compute the predicted output for test_x
    pred_y = sess.run(tf.nn.softmax(y1), feed_dict={x1: test_features,keep_probab:1.0} )
#     max=max(pred_y[0])

    top_five=sess.run(tf.nn.top_k(pred_y,k=1))
    print(top_five)

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

softmax=tf.nn.softmax(logits)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver=tf.train.import_meta_graph('lenet.meta')
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    probability=sess.run(tf.nn.softmax(logits),{x:test_image_array,y:image_labels, keep_prob:1.0})
    top_five=sess.run(tf.nn.top_k(probability,k=1))
    print(top_five)

In [ ]:
def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")